In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
torch.manual_seed(42)

<span style="font-size: 15px;">Neural network layers are the fundamental building blocks that transform input data into meaningful representations. Each layer type is designed for specific data structures and tasks. The **shape of the input and output tensors** plays a crucial role in understanding how these layers operate.

In what follows, we investigate the most common layer types in PyTorch:

1. **nn.Linear**: Fully connected layers for general-purpose transformations
2. **nn.Conv1d**: 1D convolutions for sequential/temporal data
3. **nn.Conv2d**: 2D convolutions for image data
4. **nn.Conv3d**: 3D convolutions for volumetric/video data

Throughout this notebook:
- $B$ denotes the batch size
- $C_{\text{in}}$ and $C_{\text{out}}$ denote input and output channels/features
- Spatial dimensions are denoted by $L$ (length), $H$ (height), $W$ (width), $D$ (depth)
</span>

**Overview**

| Layer | Input Shape | Output Shape | Primary Use Cases | Key Parameters |
|-------|-------------|--------------|-------------------|----------------|
| **nn.Linear** | $(*, H_{\text{in}})$ | $(*, H_{\text{out}})$ | MLPs, classification heads, dense connections, Transformer projections | `in_features`, `out_features`, `bias` |
| **nn.Conv1d** | $(B, C_{\text{in}}, L)$ | $(B, C_{\text{out}}, L_{\text{out}})$ | Time series, audio, text (1D sequences) | `in_channels`, `out_channels`, `kernel_size`, `stride`, `padding` |
| **nn.Conv2d** | $(B, C_{\text{in}}, H, W)$ | $(B, C_{\text{out}}, H_{\text{out}}, W_{\text{out}})$ | Images, feature maps, 2D spatial data | `in_channels`, `out_channels`, `kernel_size`, `stride`, `padding`, `dilation`, `groups` |
| **nn.Conv3d** | $(B, C_{\text{in}}, D, H, W)$ | $(B, C_{\text{out}}, D_{\text{out}}, H_{\text{out}}, W_{\text{out}})$ | Videos, medical imaging (CT/MRI), 3D point clouds | `in_channels`, `out_channels`, `kernel_size`, `stride`, `padding`, `dilation`, `groups` |

Detailed explanations of each layer, including mathematical formulations and implementation examples, follow below.

## nn.Linear

<span style="font-size: 15px;">


### Where is it used?

The `nn.Linear` layer (also known as a fully connected or dense layer) applies a linear transformation to the incoming data. It is the most fundamental building block in neural networks and is used in:

- **Multi-Layer Perceptrons (MLPs)**: The backbone of simple feedforward networks
- **Classification heads**: Final layers that map features to class logits
- **Transformer architectures**: Q, K, V projections and feed-forward networks
- **Autoencoders**: Encoding and decoding dense representations
- **Regression tasks**: Mapping features to continuous outputs

### Input and Output Shapes

- **Input**: $(*, H_{\text{in}})$ where $*$ means any number of dimensions and $H_{\text{in}}$ is the number of input features
- **Output**: $(*, H_{\text{out}})$ where all dimensions except the last remain unchanged

**Important**: The linear transformation is applied to the **last dimension** only.

### Default Arguments

```python
nn.Linear(in_features, out_features, bias=True, device=None, dtype=None)
```

- `in_features` (int): Size of each input sample (required)
- `out_features` (int): Size of each output sample (required)
- `bias` (bool): If `True`, adds a learnable bias. Default: `True`
- 
### Mathematical Formulation

The linear layer applies the following transformation:

$$
{y_{\rm out}}^{i,j,\cdots, k} = \omega_{k z} \, {x_{\rm in}}^{i,j,\cdots, z} + b^{k}
$$


where the indices "$i,j,\cdots$" represent an arbitrary dimension, i.e., $*$. This makes the layer **batch-agnostic**. As we see,
- $x$ is the input tensor of shape $(*, H_{\text{in}})$
- $\omega$ is the weight matrix of shape $(H_{\text{out}}, H_{\text{in}})$
- $b$ is the bias vector of shape $(H_{\text{out}})$
- $y$ is the output tensor of shape $(*, H_{\text{out}})$
- **Notice**, that, all elements in the arbirary dimension "$i,j,\cdots$" get exactly the same biases and weights.




### Weight Initialization

Weights are initialized from $\mathcal{U}(-\sqrt{k}, \sqrt{k})$ where $k = \frac{1}{H_{\text{in}}}$

</span>

In [27]:
# Basic nn.Linear usage
# Create a linear layer that maps 20 input features to 30 output features
linear = nn.Linear(20, 30)

# Check the weight and bias shapes
print(f"Weight shape: {linear.weight.shape}")  # (out_features, in_features)
print(f"Bias shape: {linear.bias.shape}")      # (out_features,)

Weight shape: torch.Size([30, 20])
Bias shape: torch.Size([30])


In [28]:
# Example 1: Simple 2D input (batch_size, in_features)
batch_size = 128
in_features = 20
out_features = 30

linear = nn.Linear(in_features, out_features)
models_weights = linear.weight
models_biases = linear.bias
x = torch.randn(batch_size, in_features)
output = linear(x)

print(f"Input shape: {x.shape}")
print(f"Output shape: {output.shape}")

# Lets now try to reproduce the output manually:

output_manual = torch.einsum('bi,oi->bo', x, models_weights) + models_biases.unsqueeze(0).expand(batch_size, -1)
print(f'Outputs match: {torch.all(output_manual==output).item()}')

Input shape: torch.Size([128, 20])
Output shape: torch.Size([128, 30])
Outputs match: True


In [29]:
# Example 3: Without bias
linear_no_bias = nn.Linear(20, 30, bias=False)
print(f"Has bias: {linear_no_bias.bias is not None}")

# Verify the transformation manually
x = torch.randn(5, 20)
output_layer = linear_no_bias(x)
output_manual = x @ linear_no_bias.weight.T  # y = x @ W^T

print(f"Outputs match: {torch.allclose(output_layer, output_manual)}")

Has bias: False
Outputs match: True


## Convolutional layers

Before we start discussing the types of convolutional layers, we briefly discuss some of the important padding types appearing in these layers:



| Padding | Conv1d | Conv2d | Conv3d | Meaning |
|---------|--------|--------|--------|---------|
| `padding=0` | `0` | `(0,0)` | `(0,0,0)` | No padding, output shrinks |
| `padding='valid'` | `0` | `(0,0)` | `(0,0,0)` | Equivalent to `padding=0` |
| `padding='same'` | auto | auto | auto | Output size = Input size (requires `stride=1`) |
| `padding=k//2` | `k//2` | `(k//2, k//2)` | `(k//2, k//2, k//2)` | Preserves size for odd kernel $k$ with `stride=1`, `dilation=1` |

**Common recipe to preserve spatial dimensions:**

For `kernel_size=k`, `stride=1`, `dilation=1`:
```python
# These are equivalent:
padding = 'same'
padding = k // 2   # only works for odd k
```

**Examples preserving size:**

| Kernel | Padding |
|--------|---------|
| 3 | 1 |
| 5 | 2 |
| 7 | 3 |

**General formula:** `padding = (kernel_size - 1) // 2` for odd kernels with `stride=1`, `dilation=1`.

### nn.Conv1d

<span style="font-size: 15px;">

#### Where is it used?

The `nn.Conv1d` layer applies a 1D convolution over an input signal composed of several input planes. It is primarily used for:

- **Time series analysis**: Stock prices, sensor data, weather patterns
- **Audio processing**: Speech recognition, music classification
- **Text/NLP**: Character-level or word-level sequence modeling
- **Signal processing**: Any 1D sequential data

#### Input and Output Shapes

- **Input**: $(B, C_{\text{in}}, L_{\text{in}})$ where:
  - $B$ = batch size
  - $C_{\text{in}}$ = number of input channels
  - $L_{\text{in}}$ = length of the input sequence

- **Output**: $(B, C_{\text{out}}, L_{\text{out}})$ where:
  - $C_{\text{out}}$ = number of output channels (number of filters)
  - $L_{\text{out}} = \left\lfloor\frac{L_{\text{in}} + 2 \times \text{padding} - \text{dilation} \times (\text{kernel\_size} - 1) - 1}{\text{stride}} + 1\right\rfloor$

#### Default Arguments

```python
nn.Conv1d(in_channels, out_channels, kernel_size, stride=1, padding=0, 
          dilation=1, groups=1, bias=True, padding_mode='zeros')
```

- `in_channels` (int): Number of channels in the input (required)
- `out_channels` (int): Number of channels produced by the convolution (required)
- `kernel_size` (int or tuple): Size of the convolving kernel (required)
- `stride` (int): Stride of the convolution. Default: `1`
- `padding` (int or str): Zero-padding added to both sides. Default: `0`. Can be `'same'` or `'valid'`
- `dilation` (int): Spacing between kernel elements. Default: `1`
- `groups` (int): Number of blocked connections. Default: `1`
- `bias` (bool): If `True`, adds a learnable bias. Default: `True`
- `padding_mode` (str): `'zeros'`, `'reflect'`, `'replicate'`, or `'circular'`. Default: `'zeros'`

#### Weight Shape

The weight tensor has shape: $(C_{\text{out}}, \frac{C_{\text{in}}}{\text{groups}}, \text{kernel\_size})$



#### Mathematical Formulation

For an input of size $(B, C_{\text{in}}, L)$ and output of size $(B, C_{\text{out}}, L_{\text{out}})$, the convolution from $x_{\rm in}[b, c, i]$ to $y_{\rm out}[b, c, i] $ is performed as follows:


* For `groups`$ = 1$:
$$
\begin{aligned}
y_{\rm out}[b, c_{out}, i] = \sum_{c_{\rm in}=0}^{C_{\rm in} -1} \sum_{k=0}^{K -1}~{\tilde x}_{\rm in}[b,~ c_{\rm in},~ s*i+~d*k] \omega[c_{\rm out}, c_{\rm in}, k] + \beta[c_{\rm out}]
\end{aligned}
$$

* For `groups`$ > 1$ and, `in_channels % groups`$~=~$ `out_channels % groups`$=0$:
$$
\begin{aligned}
y_{\rm out}[b, c_{out}, i] = \sum_{j=0}^{C_{\rm in}/g -1} \sum_{k=0}^{K -1}~{\tilde x}_{\rm in}[b,~ m(c_{\rm out})*\frac{C_{\rm in}}{g} + j,~ s*i+~d*k] \omega[c_{\rm out}, j, k] + \beta[c_{\rm out}]
\end{aligned}
$$

where ${\tilde x}_{\rm in}$ is the padded input tensor, it is simply equal to $x_{\rm in}$ if padding is zero. Moreover, $g = $ `groups`, $s = $ `stride`, $d = $ `dilation` , and $m(c_{\rm out}) = {\rm floor}( c_{out} * g / C_{out} )$.



**Remark**: When `groups` $=1$, each filter combines information from all input channels. Otherwise, channels are processed in isolated groups, reducing computation and parameters but limiting cross-channel interactio.

</span>

In [30]:
# Basic nn.Conv1d usage
# Input: 16 channels, Output: 33 channels, kernel size: 3
conv1d = nn.Conv1d(in_channels=16, out_channels=33, kernel_size=3)

print(f"Weight shape: {conv1d.weight.shape}")  # (out_channels, in_channels, kernel_size)
print(f"Bias shape: {conv1d.bias.shape}")      # (out_channels,)

Weight shape: torch.Size([33, 16, 3])
Bias shape: torch.Size([33])


In [31]:
# Example 1: Basic convolution without padding
batch_size = 20
in_channels = 16
out_channels = 33
kernel_size = 3
L_in = 50  # Input sequence length

conv1d = nn.Conv1d(in_channels, out_channels, kernel_size)
x = torch.randn(batch_size, in_channels, L_in)
output = conv1d(x)

# Calculate expected output length: L_out = (L_in + 2*padding - dilation*(kernel_size-1) - 1) / stride + 1
L_out_expected = (L_in + 2*0 - 1*(kernel_size-1) - 1) // 1 + 1

print(f"Input shape: {x.shape}")
print(f"Output shape: {output.shape}")
print(f"Expected L_out: {L_out_expected}")

Input shape: torch.Size([20, 16, 50])
Output shape: torch.Size([20, 33, 48])
Expected L_out: 48


In [32]:
# Example 2: Convolution with padding to maintain spatial dimension
# For kernel_size=3, padding=1 maintains the length (with stride=1)
conv1d_same = nn.Conv1d(in_channels=16, out_channels=33, kernel_size=3, padding=1)
x = torch.randn(20, 16, 50)
output = conv1d_same(x)

print(f"Input shape: {x.shape}")
print(f"Output shape: {output.shape}")
print(f"Length preserved: {x.shape[2] == output.shape[2]}")

Input shape: torch.Size([20, 16, 50])
Output shape: torch.Size([20, 33, 50])
Length preserved: True


In [33]:
# Example 3: Using padding='same' (requires stride=1)
conv1d_same_str = nn.Conv1d(in_channels=16, out_channels=33, kernel_size=5, padding='same')
x = torch.randn(20, 16, 50)
output = conv1d_same_str(x)

print(f"Input shape: {x.shape}")
print(f"Output shape: {output.shape}")

Input shape: torch.Size([20, 16, 50])
Output shape: torch.Size([20, 33, 50])


In [38]:
# Example 4: Convolution with stride > 1 (downsampling)
conv1d_stride = nn.Conv1d(in_channels=16, out_channels=33, kernel_size=3, stride=2)
x = torch.randn(20, 16, 50)
output = conv1d_stride(x)

# L_out = (50 + 2*0 - 1*(3-1) - 1) // 2 + 1 = (50 - 2 - 1) // 2 + 1 = 47//2 + 1 = 23 + 1 = 24
print(f"Input shape: {x.shape}")
print(f"Output shape: {output.shape}")

Input shape: torch.Size([20, 16, 50])
Output shape: torch.Size([20, 33, 24])


In [39]:
# Example 5: Dilated convolution (for larger receptive field)
conv1d_dilated = nn.Conv1d(in_channels=16, out_channels=33, kernel_size=3, dilation=2)
x = torch.randn(20, 16, 50)
output = conv1d_dilated(x)

# L_out = (50 + 2*0 - 2*(3-1) - 1) / 1 + 1 = (50 - 4 - 1) + 1 = 46
print(f"Input shape: {x.shape}")
print(f"Output shape: {output.shape}")
print(f"Effective kernel size with dilation=2: {2*(3-1)+1}")

Input shape: torch.Size([20, 16, 50])
Output shape: torch.Size([20, 33, 46])
Effective kernel size with dilation=2: 5


### nn.Conv2d